In [9]:
import pandas as pd
from sqlalchemy import create_engine, exc
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Database credentials and connection details
db_config = {
    "user": os.getenv("DB_USER", "Buddy"),
    "password": os.getenv("DB_PASSWORD", "Hirusha_22311"),
    "host": os.getenv("DB_HOST", "localhost"),
    "database": os.getenv("DB_NAME", "mydb")
}

# Paths to CSV files
customer_csv_path = r"C:\Users\hirus\Desktop\Delivergate Project\customers.csv"
orders_csv_path = r"C:\Users\hirus\Desktop\Delivergate Project\order.csv"

# Check if CSV files exist
if not os.path.exists(customer_csv_path):
    logging.error(f"Customer CSV file not found at {customer_csv_path}")
    raise FileNotFoundError(f"Customer CSV file not found at {customer_csv_path}")

if not os.path.exists(orders_csv_path):
    logging.error(f"Orders CSV file not found at {orders_csv_path}")
    raise FileNotFoundError(f"Orders CSV file not found at {orders_csv_path}")

# Establishing connection using SQLAlchemy
try:
    # Creating a SQLAlchemy engine
    engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

    # Reading customer data
    customers_df = pd.read_csv(customer_csv_path)
    # Creating or replacing 'customers' table and loading data
    customers_df.to_sql('customers', con=engine, if_exists='replace', index=False)
    logging.info("Customer data imported successfully.")

    # Reading order data
    orders_df = pd.read_csv(orders_csv_path)
    # Creating or replacing 'orders' table and loading data
    orders_df.to_sql('orders', con=engine, if_exists='replace', index=False)
    logging.info("Order data imported successfully.")

except exc.SQLAlchemyError as e:
    logging.error("An error occurred while connecting to the database or inserting data: %s", e)
except FileNotFoundError as e:
    logging.error(e)
except Exception as e:
    logging.error("An unexpected error occurred: %s", e)
finally:
    # Close the connection
    if 'engine' in locals():
        engine.dispose()
        logging.info("Database connection closed.")

INFO:root:Customer data imported successfully.
INFO:root:Order data imported successfully.
INFO:root:Database connection closed.


In [11]:
from sqlalchemy import create_engine, exc, text
from sqlalchemy.orm import sessionmaker
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Database configuration
db_config = {
    "user": "Buddy",
    "password": "Hirusha_22311",
    "host": "localhost",
    "database": "mydb"
}

def create_db_connection():
    try:
        # Create a SQLAlchemy engine
        connection_string = f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"
        engine = create_engine(connection_string, pool_recycle=3600)
        
        # Test the connection
        engine.connect()
        logger.info("Successfully connected to the database.")
        
        return engine

    except exc.SQLAlchemyError as e:
        logger.error("Database connection failed.", exc_info=True)
        print("An error occurred while connecting to the database.")
        return None

def column_exists(session, table_name, column_name):
    check_column_query = text("""
        SELECT COUNT(*)
        FROM information_schema.columns
        WHERE table_schema = :database
        AND table_name = :table_name
        AND column_name = :column_name;
    """)
    result = session.execute(check_column_query, {
        "database": db_config["database"],
        "table_name": table_name,
        "column_name": column_name
    }).scalar()
    return result > 0

def modify_database_columns(session):
    try:
        # Set isolation level to ensure immediate visibility of changes
        session.execute(text("SET SESSION TRANSACTION ISOLATION LEVEL READ COMMITTED;"))
        
        # Rename column 'name' to 'customer_name' in the 'customers' table
        rename_customer_name = text("ALTER TABLE customers CHANGE COLUMN name customer_name VARCHAR(255);")
        session.execute(rename_customer_name)
        logger.info("Renamed 'name' to 'customer_name' in 'customers' table.")

        # Rename column 'order' to 'order_id' in the 'orders' table
        rename_order_id = text("ALTER TABLE orders CHANGE COLUMN `id` order_id INT;")
        session.execute(rename_order_id)
        logger.info("Renamed 'id' to 'order_id' in 'orders' table.")

        # Add 'order_date' if it does not exist
        if not column_exists(session, 'orders', 'order_date'):
            add_order_date = text("ALTER TABLE orders ADD COLUMN order_date DATE;")
            session.execute(add_order_date)
            logger.info("Added 'order_date' column to 'orders' table.")
        else:
            logger.info("'order_date' column already exists in 'orders' table.")

        # Populate 'order_date' with the date part of 'created_at'
        populate_order_date = text("UPDATE orders SET order_date = DATE(created_at);")
        session.execute(populate_order_date)
        logger.info("Populated 'order_date' column with data from 'created_at'.")

    except exc.SQLAlchemyError as e:
        logger.error("An error occurred while modifying the database columns.", exc_info=True)
        print("An error occurred during column modification.")

# Main execution
engine = create_db_connection()
if engine:
    Session = sessionmaker(bind=engine)
    session = Session()
    
    try:
        modify_database_columns(session)
        session.commit()  # Commit the transaction
    except Exception as e:
        session.rollback()  # Rollback if there's an error
        logger.error("Transaction failed", exc_info=True)
    finally:
        session.close()  # Close the session

    engine.dispose()
    print("Database connection closed.")


INFO:__main__:Successfully connected to the database.
INFO:__main__:Renamed 'name' to 'customer_name' in 'customers' table.
INFO:__main__:Renamed 'id' to 'order_id' in 'orders' table.
INFO:__main__:Added 'order_date' column to 'orders' table.
INFO:__main__:Populated 'order_date' column with data from 'created_at'.


Database connection closed.


In [12]:
from sqlalchemy import create_engine, exc
from sqlalchemy.orm import sessionmaker
import logging

# Configure logging for error tracking
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Database credentials and connection details
db_config = {
    "user": "Buddy",
    "password": "Hirusha_22311",
    "host": "localhost",
    "database": "mydb"
}

# Function to connect to the MySQL database
def create_db_connection():
    try:
        # SQLAlchemy connection string
        connection_string = f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"
        
        # Create an SQLAlchemy engine with the connection string
        engine = create_engine(connection_string, pool_recycle=3600)
        
        # Testing the connection
        connection = engine.connect()
        logger.info("Successfully connected to the database.")
        
        # Close test connection
        connection.close()
        
        # Return the engine to be used for database operations
        return engine

    except exc.SQLAlchemyError as e:
        # Log the error with details and provide a user-friendly message
        logger.error("Database connection failed.", exc_info=True)
        print("An error occurred while connecting to the database. Please check your configuration and permissions.")
        return None

# Use the connection function
engine = create_db_connection()

# Example of using a session for secure transactions
if engine:
    Session = sessionmaker(bind=engine)
    session = Session()
    # You can now use 'session' for database operations
    # Example:
    # session.execute("SELECT 1")
    
    # Close the session when done
    session.close()

INFO:__main__:Successfully connected to the database.


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
import datetime as dt

# Database configuration
db_config = {
    "user": "Buddy",
    "password": "Hirusha_22311",
    "host": "localhost",
    "database": "mydb"
}

# Create a database connection
connection_string = f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"
engine = create_engine(connection_string)

# Function to load data
def load_data():
    query = """
    SELECT o.order_id, o.order_date, o.total_amount, c.customer_id, c.customer_name
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    """
    df = pd.read_sql(query, engine)
    df['order_date'] = pd.to_datetime(df['order_date'])  # Ensure 'order_date' is in datetime format
    return df

# Load data
data = load_data()

# Check column names to ensure we have 'order_id', 'customer_id'
print("Columns in the dataset:")
print(data.columns)

# Feature engineering: Creating 'is_repeat_customer' column
customer_order_counts = data.groupby('customer_id')['order_id'].nunique()
data['is_repeat_customer'] = data['customer_id'].map(customer_order_counts > 1).astype(int)

# Verify the first few rows to check if the 'is_repeat_customer' column exists
print("First few rows with the 'is_repeat_customer' column:")
print(data.head())

# Aggregate data to get total revenue and total orders for each customer
customer_data = data.groupby('customer_id').agg(
    total_orders=('order_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
).reset_index()

# Check if 'is_repeat_customer' is still in the dataset
customer_data['is_repeat_customer'] = data.groupby('customer_id')['is_repeat_customer'].first().values

# Display customer data
print("Customer Data with Total Orders, Total Revenue, and Repeat Customer Status:")
print(customer_data.head())

# Define features (X) and target (y)
X = customer_data[['total_orders', 'total_revenue']]
y = customer_data['is_repeat_customer']

# Split data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (important for logistic regression)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check if there is sufficient data
if len(X_train) > 0 and len(X_test) > 0:
    print("Sufficient data for training.")
else:
    print("Not enough data for training. Please ensure a larger dataset.")

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)

# Display the accuracy
print(f"Accuracy of the Logistic Regression Model: {accuracy * 100:.2f}%")


Columns in the dataset:
Index(['order_id', 'order_date', 'total_amount', 'customer_id',
       'customer_name'],
      dtype='object')
First few rows with the 'is_repeat_customer' column:
   order_id order_date  total_amount  customer_id    customer_name  \
0     13392 2024-10-14           425         1251   Isuri Liyanage   
1     13393 2024-10-14          1650         1251   Isuri Liyanage   
2     13394 2024-10-14          1365          468  Hasitha Walpola   
3     13395 2024-10-14           525         1251   Isuri Liyanage   
4     13396 2024-10-14           525          468  Hasitha Walpola   

   is_repeat_customer  
0                   1  
1                   1  
2                   1  
3                   1  
4                   1  
Customer Data with Total Orders, Total Revenue, and Repeat Customer Status:
   customer_id  total_orders  total_revenue  is_repeat_customer
0            8            11          27500                   1
1            9             2           6797